In [3]:
!pip install Faker

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 15.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Leozinchan69\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from faker import Faker
import random
import os
import pandas as pd

In [10]:
fake = Faker()
NUM_LINHAS = 1_000_000  # colocar 1_000_000_000 depois de testar
BATCH_SIZE = 1_000_000
CAMINHO_PARQUET = os.path.abspath("transacoes_duck_bilhao2.parquet")

def gerar_lote(inicio, tamanho):
    amounts = np.round(np.random.uniform(5.00, 5000.00, size=tamanho), 2).tolist()

    merchant_categories = [
        "Supermercado", "Restaurante", "Varejo Online", "Posto de Combustível",
        "Farmácia", "Serviços de Streaming", "Transporte", "Educação",
        "Entretenimento", "Viagens", "Saúde", "Casa e Jardim"
    ]

    return pa.table({
        "transaction_id": [int(x) for x in np.arange(inicio, inicio + tamanho)],
        "customer_id": [int(x) for x in np.random.randint(1000, 1_000_000_000, size=tamanho)],
        "transaction_date": [fake.date_between(start_date='-2y', end_date='today').isoformat() for _ in range(tamanho)],
        "transaction_amount": amounts,
        "merchant_category": [random.choice(merchant_categories) for _ in range(tamanho)],
        "payment_method": [random.choice(["PIX", "Cartão de Crédito", "Cartão de Débito", "Dinheiro"]) for _ in range(tamanho)],
    })

print("📦 Iniciando geração por lotes e escrita incremental com PyArrow...")

# Writer Parquet
writer = None
contador = 1

while contador <= NUM_LINHAS:
    tamanho = min(BATCH_SIZE, NUM_LINHAS - contador + 1)
    print(f"🔄 Gerando e gravando linhas {contador} até {contador + tamanho - 1}")
    tabela = gerar_lote(contador, tamanho)

    if writer is None:
        writer = pq.ParquetWriter(CAMINHO_PARQUET, tabela.schema, compression='snappy')

    writer.write_table(tabela)
    contador += tamanho

if writer:
    writer.close()

print(f"✅ Arquivo Parquet gerado com sucesso: {CAMINHO_PARQUET}")


📦 Iniciando geração por lotes e escrita incremental com PyArrow...
🔄 Gerando e gravando linhas 1 até 1000000
✅ Arquivo Parquet gerado com sucesso: e:\engenharia_de_dados\env\transacoes_duck_bilhao2.parquet


In [4]:
df = pd.read_parquet("transacoes_duck_bilhao.parquet", engine="pyarrow")

print(df.tail(10))

        transaction_id  customer_id transaction_date  transaction_amount  \
999990          999991    974387932       2025-05-05             3337.01   
999991          999992    303754950       2025-01-02             1516.54   
999992          999993    673118603       2023-08-04              970.41   
999993          999994    928245567       2023-11-21             3402.60   
999994          999995    477578555       2024-05-28             1667.11   
999995          999996    699100455       2024-03-18             2889.92   
999996          999997    472102171       2024-12-30             3320.17   
999997          999998    763156666       2024-12-18              676.74   
999998          999999    595243340       2024-04-04             4164.29   
999999         1000000    765939637       2024-08-30              140.12   

            merchant_category     payment_method  
999990          Varejo Online           Dinheiro  
999991   Posto de Combustível  Cartão de Crédito  
999992    